In [82]:
import pandas as pd
import numpy as np
import pickle
import gc
import hyperdash as hd

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#'store_nbr', 'n_city', 'n_state', 'n_type', 'cluster', 'item_nbr', 'n_family', 'class', 'perishable'

In [76]:
df_stores.store_nbr.min()-

1

In [40]:
unit_mean, unit_std = pd.read_csv('data/mean_std.csv', index_col=0).T[0]

In [2]:
df_stores = pd.read_csv(
    'data/num_stores.csv.gz',
     dtype={
         'store_nbr': np.uint8,
         'n_city': np.uint8,
         'n_state': np.uint8,
         'n_type': np.uint8,
         'cluster': np.uint8
     }

)
df_items = pd.read_csv(
    'data/num_items.csv.gz',
    dtype={
        'item_nbr': np.int32,
        'n_family': np.uint8,
        'class': np.int16,
        'perishable': np.int8,
    }
)
for stores_col in ['n_city', 'n_state', 'n_type', 'cluster']:
    df_stores[stores_col] = df_stores[stores_col] - df_stores[stores_col].min()
    
for items_col in ['n_family', 'class', 'perishable']:
    df_items[items_col] = df_items[items_col] - df_items[items_col].min()

In [4]:
%%time
df = pd.read_csv(
    'data/ts.csv.gz',
    parse_dates=[0],
    #nrows=1000000,
    dtype={
        'item_nbr': np.int32,
        'store_nbr': np.int8,
        'unit_sales': np.float32,
        'onpromotion': np.int8,
        'holiday': np.int8,
        'weekend': np.int8,
        'waged_day': np.int8,
        'dow_0': np.int8,
        'dow_1': np.int8,
        'dow_2': np.int8,
        'dow_3': np.int8,
        'dow_4': np.int8,
        'dow_5': np.int8,
        'dow_6': np.int8,
    }
)
df.head()

CPU times: user 5min 41s, sys: 9.91 s, total: 5min 51s
Wall time: 5min 8s


In [5]:
%%time
ts_columns = df.columns[3:]
      
attr_cols = [
    'store_nbr', 'n_city', 'n_state', 'n_type', 'cluster',
    'item_nbr', 'n_family', 'class',
    #'perishable'
]

df_pivot = df.pivot_table(
    index=['store_nbr', 'item_nbr'],
    columns=['date'],
    values=ts_columns
).reset_index()

df_pivot = df_pivot.merge(df_items, on='item_nbr')
df_pivot['store_nbr'] = df_pivot[('store_nbr', '')]
df_pivot = df_pivot.merge(df_stores, on='store_nbr')

/home/dmitry/miniconda3/envs/tf_intel/lib/python3.5/site-packages/pandas/core/reshape/merge.py:551: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


CPU times: user 12min 55s, sys: 2min 27s, total: 15min 22s
Wall time: 12min 9s


In [6]:
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210654 entries, 0 to 210653
Columns: 9887 entries, item_nbr to cluster
dtypes: float64(823), int16(1), int64(4), int8(9054), uint8(5)
memory usage: 3.1 GB


In [103]:
def get_date_cols(date, history=20, predict_days=16, ts_columns=ts_columns):
                  #date, days=1, attr_cols=attr_columns_wo_means, ts_cols=ts_columns, attr=True):
    
    if type(date) != pd.Timestamp:
        date = pd.to_datetime(date)
        
    X_start_date = date - pd.Timedelta('{} days'.format(history-1))
    #X_end_date = date
    y_start_date = date + pd.Timedelta('{} days'.format(1))
    #y_end_date = date + pd.Timedelta('{} days'.format(predict_days))

    X_cols, y_cols, y_day_attr_cols = [], [], []
    
    for d in pd.date_range(X_start_date, periods=history, freq='D'):
        for elem in ts_columns:
            X_cols.append((elem, d))
            
    for d in pd.date_range(y_start_date, periods=predict_days, freq='D'):
        y_cols.append(('unit_sales_scaled', d))
        for elem in ts_columns[1:]:
            y_day_attr_cols.append((elem, d))
            
    return X_cols, y_cols, y_day_attr_cols



X_cols, y_cols, y_day_attr_cols = get_date_cols('2017-08-15', predict_days=5)
    
df_pivot.head().loc[:, y_day_attr_cols]

,"(onpromotion, 2017-08-16 00:00:00)","(holiday, 2017-08-16 00:00:00)","(weekend, 2017-08-16 00:00:00)","(waged_day, 2017-08-16 00:00:00)","(dow_0, 2017-08-16 00:00:00)","(dow_1, 2017-08-16 00:00:00)","(dow_2, 2017-08-16 00:00:00)","(dow_3, 2017-08-16 00:00:00)","(dow_4, 2017-08-16 00:00:00)","(dow_5, 2017-08-16 00:00:00)",...,"(holiday, 2017-08-20 00:00:00)","(weekend, 2017-08-20 00:00:00)","(waged_day, 2017-08-20 00:00:00)","(dow_0, 2017-08-20 00:00:00)","(dow_1, 2017-08-20 00:00:00)","(dow_2, 2017-08-20 00:00:00)","(dow_3, 2017-08-20 00:00:00)","(dow_4, 2017-08-20 00:00:00)","(dow_5, 2017-08-20 00:00:00)","(dow_6, 2017-08-20 00:00:00)"
0,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [137]:
def get_random_train_test(df_pivot,
        date, window=21, size=2000, history=1, predict_days=16, epochs=2,
        shuffle_dates=True, shuffle_indexes=True, attr_cols=attr_cols, ts_columns=ts_columns):
    
    num_items = df_pivot.shape[0]
    
    date = pd.to_datetime(date)
    start_window =  date - pd.Timedelta('{} days'.format(window))
    end_date = date
    
    dates = pd.date_range(start_window, end_date)
    
    patches = []
    #end_X_date = end_date - pd.Timedelta('{} days'.format(label_dates))
    if shuffle_dates and shuffle_indexes:
        permutated_dates = np.random.permutation(dates)
        permutated_indx = np.random.permutation(num_items)   
        for epoch in range(epochs):
            for i in range(num_items//size+1):
                s = size * i
                e = size * (i+1)
                indexes = permutated_indx[s:e]

                for date in permutated_dates:
                    patches.append([indexes, date])

        patches = np.random.permutation(patches)
        
    elif not shuffle_dates and shuffle_indexes:
        permutated_indx = np.random.permutation(num_items)
        for date in dates:
            for epoch in range(epochs):
                for i in range(num_items//size+1):
                    s = size * i
                    e = size * (i+1)
                    indexes = permutated_indx[s:e]
                    patches.append([indexes, date])

    for indexes, date in patches:
        df_pivot_slice = df_pivot.iloc[indexes]
        X_cols, y_cols, y_day_attr_cols = get_date_cols(
            date, history=history, predict_days=predict_days, ts_columns=ts_columns
        )

        X = np.array(
            df_pivot_slice.loc[:, X_cols]
        ).reshape([-1, history, len(ts_columns)])

        y_day_attr = np.array(
            df_pivot_slice.loc[:, y_day_attr_cols]
        ).reshape([-1, predict_days, len(ts_columns)-1])
        
        y = np.array(df_pivot_slice.loc[:, y_cols])
        features = [X, y_day_attr, y]
        for feature in attr_cols:
            features.append(
                np.array(df_pivot_slice.loc[:, feature])
            )
        for i in range(len(indexes)):
            yield tuple([elem[i] for elem in features])

tmp = get_random_train_test(df_pivot, '2017-07-15', window=0, history=90, predict_days=5)
tmp1 = next(tmp)
print(tmp1[0].shape, tmp1[1].shape, tmp1[2].shape, tmp1[3].shape)

(90, 12) (5, 11) (5,) ()


In [147]:
for i in range(2000):
    next(tmp)

In [148]:
next(tmp)

(array([[-0.71689729,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-0.71689729,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-0.71689729,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [-0.71689729,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-0.71689729,  0.        ,  0.        , ...,  1.        ,
          0.        ,  0.        ],
        [-0.71689729,  0.        ,  0.        , ...,  0.        ,
          1.        ,  0.        ]]), array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]], dtype=int8), array([-0.71689729, -0.71689729, -0.71689729, -0.71689729, -0.71689729]), 36, 11, 6, 4, 9, 1960806, 30, 106)

In [150]:
def get_validation(df_pivot,
        date, history=1, predict_days=16, attr_cols=attr_cols, ts_columns=ts_columns):
    
    X_cols, y_cols, y_day_attr_cols = get_date_cols(
        date, history=history, predict_days=predict_days, ts_columns=ts_columns
    )

    X = np.array(
        df_pivot.loc[:, X_cols]
    ).reshape([-1, history, len(ts_columns)])
    
    y_day_attr = np.array(
        df_pivot.loc[:, y_day_attr_cols]
    ).reshape([-1, predict_days, len(ts_columns)-1])

    y = np.array(df_pivot.loc[:, y_cols])
    features = [X, y_day_attr, y]
    for feature in attr_cols:
        features.append(
            np.array(df_pivot.loc[:, feature])
        )

    return features

tmp = get_validation(df_pivot, '2017-07-15', history=90, predict_days=5)
print(tmp[0].shape, tmp[1].shape, tmp[2].shape, tmp[3].shape)

(210654, 90, 12) (210654, 5, 11) (210654, 5) (210654,)


In [155]:
%autoreload 2

history = 200
time_to_predict = 16

last_day_train = '2017-07-15'
window=21
epochs = 100
validation_day = pd.to_datetime(last_day_train) + pd.Timedelta('{} days'.format(time_to_predict))
batch_size = 2000
sum_W = 3574368.0/16

print(validation_day)

batch_gen = get_random_train_test(
    df_pivot,
    last_day_train,
    window=window,
    history=history,
    size=batch_size,
    predict_days=time_to_predict,
    epochs=epochs
)

val_set = get_validation(df_pivot, validation_day, history=history, predict_days=time_to_predict)

from model import RNNModel

m = RNNModel(
    history=history,
    n_days_predict=time_to_predict,
    clip_gradients=1.,
    starter_learning_rate=0.00005,
    #starter_learning_rate=0.0005,
    n_layers_rnn=1,
    rnn_size_encoder=200,
    rnn_size_decoder=200,
)
print(1)
m.build_graph(batch_gen)


try:
    hd_exp.end()
except NameError:
    pass

hd_exp = hd.Experiment('RNN fav 2')

m.train(val_set, coef=unit_std, sum_W=sum_W,
        report_every=100, validate_every=1000,
        hd_exp=hd_exp)

2017-07-31 00:00:00
1
g_step: 100 loss std/mean: 4.231199741363525 6.726279258728027
| Loss std:   4.231200 |
| Loss mean:   6.726279 |
g_step: 200 loss std/mean: 0.1736292839050293 3.69563364982605
| Loss std:   0.173629 |
| Loss mean:   3.695634 |
g_step: 300 loss std/mean: 0.19275760650634766 3.2100281715393066
| Loss std:   0.192758 |
| Loss mean:   3.210028 |
g_step: 400 loss std/mean: 0.15953347086906433 2.823206901550293
| Loss std:   0.159533 |
| Loss mean:   2.823207 |
g_step: 500 loss std/mean: 0.13961458206176758 2.5237131118774414
| Loss std:   0.139615 |
| Loss mean:   2.523713 |
g_step: 600 loss std/mean: 0.15553928911685944 2.263000726699829
| Loss std:   0.155539 |
| Loss mean:   2.263001 |
g_step: 700 loss std/mean: 0.1281239092350006 1.9955003261566162
| Loss std:   0.128124 |
| Loss mean:   1.995500 |
g_step: 800 loss std/mean: 0.1018436849117279 1.7287566661834717
| Loss std:   0.101844 |
| Loss mean:   1.728757 |
g_step: 900 loss std/mean: 0.08400782197713852 1.484

| Loss mean:   0.330104 |
g_step: 6300 loss std/mean: 0.020574580878019333 0.3292018175125122
| Loss std:   0.020575 |
| Loss mean:   0.329202 |
g_step: 6400 loss std/mean: 0.02253977209329605 0.3295978605747223
| Loss std:   0.022540 |
| Loss mean:   0.329598 |
g_step: 6500 loss std/mean: 0.01912682130932808 0.32699131965637207
| Loss std:   0.019127 |
| Loss mean:   0.326991 |
g_step: 6600 loss std/mean: 0.01783089153468609 0.3278654217720032
| Loss std:   0.017831 |
| Loss mean:   0.327865 |
g_step: 6700 loss std/mean: 0.020620770752429962 0.32686659693717957
| Loss std:   0.020621 |
| Loss mean:   0.326867 |
g_step: 6800 loss std/mean: 0.021171147003769875 0.32864639163017273
| Loss std:   0.021171 |
| Loss mean:   0.328646 |
g_step: 6900 loss std/mean: 0.022555779665708542 0.32556724548339844
| Loss std:   0.022556 |
| Loss mean:   0.325567 |
g_step: 7000 loss std/mean: 0.01940901018679142 0.3233162760734558
| Loss std:   0.019409 |
| Loss mean:   0.323316 |
	Validation NWRMSLE  :

g_step: 12400 loss std/mean: 0.017503412440419197 0.2991253137588501
| Loss std:   0.017503 |
| Loss mean:   0.299125 |
g_step: 12500 loss std/mean: 0.016813740134239197 0.29722142219543457
| Loss std:   0.016814 |
| Loss mean:   0.297221 |
g_step: 12600 loss std/mean: 0.01971697062253952 0.29886317253112793
| Loss std:   0.019717 |
| Loss mean:   0.298863 |
g_step: 12700 loss std/mean: 0.02109895460307598 0.3037976026535034
| Loss std:   0.021099 |
| Loss mean:   0.303798 |
g_step: 12800 loss std/mean: 0.01827353797852993 0.2990674376487732
| Loss std:   0.018274 |
| Loss mean:   0.299067 |
g_step: 12900 loss std/mean: 0.017160894349217415 0.2966272830963135
| Loss std:   0.017161 |
| Loss mean:   0.296627 |
g_step: 13000 loss std/mean: 0.020992135629057884 0.2997037172317505
| Loss std:   0.020992 |
| Loss mean:   0.299704 |
	Validation NWRMSLE  : 0.624498304151
| Validation NWRMSLE:   0.624498 |
	Validation NWRMSLE_5: 0.580315919601
| Validation NWRMSLE_5:   0.580316 |
g_step: 13100

g_step: 18500 loss std/mean: 0.018682189285755157 0.28877025842666626
| Loss std:   0.018682 |
| Loss mean:   0.288770 |
g_step: 18600 loss std/mean: 0.019399305805563927 0.29339611530303955
| Loss std:   0.019399 |
| Loss mean:   0.293396 |
g_step: 18700 loss std/mean: 0.0192036721855402 0.2891894280910492
| Loss std:   0.019204 |
| Loss mean:   0.289189 |
g_step: 18800 loss std/mean: 0.020368266850709915 0.29396095871925354
| Loss std:   0.020368 |
| Loss mean:   0.293961 |
g_step: 18900 loss std/mean: 0.021514451131224632 0.29109400510787964
| Loss std:   0.021514 |
| Loss mean:   0.291094 |
g_step: 19000 loss std/mean: 0.018605323508381844 0.29180335998535156
| Loss std:   0.018605 |
| Loss mean:   0.291803 |
	Validation NWRMSLE  : 0.610981746492
| Validation NWRMSLE:   0.610982 |
	Validation NWRMSLE_5: 0.539070289923
| Validation NWRMSLE_5:   0.539070 |
g_step: 19100 loss std/mean: 0.016261683776974678 0.2858944237232208
| Loss std:   0.016262 |
| Loss mean:   0.285894 |
g_step: 1

g_step: 24600 loss std/mean: 0.020335251465439796 0.285776287317276
| Loss std:   0.020335 |
| Loss mean:   0.285776 |
g_step: 24700 loss std/mean: 0.018306424841284752 0.28130003809928894
| Loss std:   0.018306 |
| Loss mean:   0.281300 |
g_step: 24800 loss std/mean: 0.015515323728322983 0.2793905436992645
| Loss std:   0.015515 |
| Loss mean:   0.279391 |
g_step: 24900 loss std/mean: 0.016941571608185768 0.2800602614879608
| Loss std:   0.016942 |
| Loss mean:   0.280060 |
g_step: 25000 loss std/mean: 0.018525414168834686 0.2796338200569153
| Loss std:   0.018525 |
| Loss mean:   0.279634 |
	Validation NWRMSLE  : 0.601123572637
| Validation NWRMSLE:   0.601124 |
	Validation NWRMSLE_5: 0.540177612758
| Validation NWRMSLE_5:   0.540178 |
g_step: 25100 loss std/mean: 0.017477314919233322 0.2794482111930847
| Loss std:   0.017477 |
| Loss mean:   0.279448 |
g_step: 25200 loss std/mean: 0.017715970054268837 0.2811070382595062
| Loss std:   0.017716 |
| Loss mean:   0.281107 |
g_step: 2530

...

| Loss mean:   0.262204 |
g_step: 73300 loss std/mean: 0.013708223588764668 0.2622600495815277
| Loss std:   0.013708 |
| Loss mean:   0.262260 |
g_step: 73400 loss std/mean: 0.013308031484484673 0.26304370164871216
| Loss std:   0.013308 |
| Loss mean:   0.263044 |
g_step: 73500 loss std/mean: 0.013970455154776573 0.2611266076564789
| Loss std:   0.013970 |
| Loss mean:   0.261127 |
g_step: 73600 loss std/mean: 0.015466885641217232 0.26182517409324646
| Loss std:   0.015467 |
| Loss mean:   0.261825 |
g_step: 73700 loss std/mean: 0.016838407143950462 0.26447081565856934
| Loss std:   0.016838 |
| Loss mean:   0.264471 |
g_step: 73800 loss std/mean: 0.01650526002049446 0.2647608518600464
| Loss std:   0.016505 |
| Loss mean:   0.264761 |
g_step: 73900 loss std/mean: 0.016497567296028137 0.2653887867927551
| Loss std:   0.016498 |
| Loss mean:   0.265389 |
g_step: 74000 loss std/mean: 0.013023100793361664 0.2637024521827698
| Loss std:   0.013023 |
| Loss mean:   0.263702 |
	Validation 

| Loss mean:   0.262222 |
g_step: 79400 loss std/mean: 0.013914469629526138 0.26050353050231934
| Loss std:   0.013914 |
| Loss mean:   0.260504 |
g_step: 79500 loss std/mean: 0.01693948172032833 0.26152968406677246
| Loss std:   0.016939 |
| Loss mean:   0.261530 |
g_step: 79600 loss std/mean: 0.01517979335039854 0.2605641484260559
| Loss std:   0.015180 |
| Loss mean:   0.260564 |
g_step: 79700 loss std/mean: 0.01588616892695427 0.2607436776161194
| Loss std:   0.015886 |
| Loss mean:   0.260744 |
g_step: 79800 loss std/mean: 0.014035578817129135 0.2593555152416229
| Loss std:   0.014036 |
| Loss mean:   0.259356 |
g_step: 79900 loss std/mean: 0.012261888943612576 0.26317471265792847
| Loss std:   0.012262 |
| Loss mean:   0.263175 |
g_step: 80000 loss std/mean: 0.017750782892107964 0.2660100758075714
| Loss std:   0.017751 |
| Loss mean:   0.266010 |
	Validation NWRMSLE  : 0.61488196109
| Validation NWRMSLE:   0.614882 |
	Validation NWRMSLE_5: 0.533345793462
| Validation NWRMSLE_5: 

| Loss mean:   0.262339 |
g_step: 85500 loss std/mean: 0.019896775484085083 0.2659187614917755
| Loss std:   0.019897 |
| Loss mean:   0.265919 |
g_step: 85600 loss std/mean: 0.013656042516231537 0.2619706690311432
| Loss std:   0.013656 |
| Loss mean:   0.261971 |
g_step: 85700 loss std/mean: 0.014910273253917694 0.2604786455631256
| Loss std:   0.014910 |
| Loss mean:   0.260479 |
g_step: 85800 loss std/mean: 0.017959600314497948 0.26262304186820984
| Loss std:   0.017960 |
| Loss mean:   0.262623 |
g_step: 85900 loss std/mean: 0.015438403002917767 0.26371443271636963
| Loss std:   0.015438 |
| Loss mean:   0.263714 |
g_step: 86000 loss std/mean: 0.01688562147319317 0.26074790954589844
| Loss std:   0.016886 |
| Loss mean:   0.260748 |
	Validation NWRMSLE  : 0.596645308006
| Validation NWRMSLE:   0.596645 |
	Validation NWRMSLE_5: 0.53378260526
| Validation NWRMSLE_5:   0.533783 |
g_step: 86100 loss std/mean: 0.015438693575561047 0.25881823897361755
| Loss std:   0.015439 |
| Loss mea

| Loss mean:   0.259747 |
g_step: 91600 loss std/mean: 0.01727166399359703 0.2603956460952759
| Loss std:   0.017272 |
| Loss mean:   0.260396 |
g_step: 91700 loss std/mean: 0.012413839809596539 0.26028746366500854
| Loss std:   0.012414 |
| Loss mean:   0.260287 |
g_step: 91800 loss std/mean: 0.01788012869656086 0.2589249908924103
| Loss std:   0.017880 |
| Loss mean:   0.258925 |
g_step: 91900 loss std/mean: 0.01423169206827879 0.26057565212249756
| Loss std:   0.014232 |
| Loss mean:   0.260576 |
g_step: 92000 loss std/mean: 0.01724722981452942 0.2629908621311188
| Loss std:   0.017247 |
| Loss mean:   0.262991 |
	Validation NWRMSLE  : 0.596144719936
| Validation NWRMSLE:   0.596145 |
	Validation NWRMSLE_5: 0.543436925038
| Validation NWRMSLE_5:   0.543437 |
g_step: 92100 loss std/mean: 0.016925148665905 0.26201632618904114
| Loss std:   0.016925 |
| Loss mean:   0.262016 |
g_step: 92200 loss std/mean: 0.017448460683226585 0.26019221544265747
| Loss std:   0.017448 |
| Loss mean:   

| Loss mean:   0.260345 |
g_step: 97700 loss std/mean: 0.016602955758571625 0.26029112935066223
| Loss std:   0.016603 |
| Loss mean:   0.260291 |
g_step: 97800 loss std/mean: 0.014255653135478497 0.25991418957710266
| Loss std:   0.014256 |
| Loss mean:   0.259914 |
g_step: 97900 loss std/mean: 0.01573876477777958 0.26192909479141235
| Loss std:   0.015739 |
| Loss mean:   0.261929 |
g_step: 98000 loss std/mean: 0.015793437138199806 0.2578538656234741
| Loss std:   0.015793 |
| Loss mean:   0.257854 |
	Validation NWRMSLE  : 0.606554866278
| Validation NWRMSLE:   0.606555 |
	Validation NWRMSLE_5: 0.531509662231
| Validation NWRMSLE_5:   0.531510 |
g_step: 98100 loss std/mean: 0.016029048711061478 0.2607165277004242
| Loss std:   0.016029 |
| Loss mean:   0.260717 |
g_step: 98200 loss std/mean: 0.014732345938682556 0.25612616539001465
| Loss std:   0.014732 |
| Loss mean:   0.256126 |
g_step: 98300 loss std/mean: 0.0141181955114007 0.2621631920337677
| Loss std:   0.014118 |
| Loss mean